Список задач, утверждённых на созвоне:
1. Разбиение по группам одинаковых названий компаний
2. Разбиение датасета на `train` и `test` части. Часть test  будет балансированной, часть train будет как полной так и сбалансированной.
3. Введение мертики map@n
4. Выделение фичей в парах слов и обучение на этих фичах классификатора



In [1]:
import pandas as pd
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [2]:
df_train = pd.read_csv("train.csv", index_col=0)
df_test = pd.read_csv("test.csv", index_col=0)

Поскольку мы будем заниматься "кластеризацией" названий, я не вижу смысла брать сбалансированный датасет. При желании, вы сами можете взять сбалансированный датасет и загрузить его в переменную `df` внутри следующей ячейки

In [3]:
df = pd.concat([df_train, df_test]).sort_index()

# 1. Разбиение на группы

In [4]:
all_company_names = list(set(list(df.name_1.values) + list(df.name_2.values)))
df_all_pairs_for_name_1 = pd.concat(
    [df, df.rename(columns={"name_1": "name_2", "name_2": "name_1"})]
).drop_duplicates()
all_clusters = []

for company_name in tqdm(all_company_names):

    # Check for company name in clusters
    company_in_clusters = company_name in [x for y in all_clusters for x in y]

    # Create new cluster:
    if not company_in_clusters:
        check_list = [company_name]
        last_df_len = 0
        while True:
            df_filter = df_all_pairs_for_name_1
            df_filter = df_filter[df_filter.name_1.isin(check_list)]
            df_filter = df_filter[df_filter.is_duplicate == 1]
            check_list = list(set(list(df_filter.name_2.values) + check_list))
            if df_filter.shape[0] > last_df_len:
                last_df_len = df_filter.shape[0]
            else:
                all_clusters.append(check_list)
                break

100%|██████████| 17814/17814 [04:45<00:00, 62.32it/s]


Видно, что есть названия которые повторяются в кластерах, но их 300 шт, к сожалению учитывая недостаток времени, придётся доаустить такою погрешность

In [5]:
flaten_clusters = [x for y in all_clusters for x in y]
print("Все названия в кластерах", len(flaten_clusters))
print("уникальные названия в кластерах", len(set(flaten_clusters)))
len(flaten_clusters) == len(set(flaten_clusters))

Все названия в кластерах 17814
уникальные названия в кластерах 17814


True

Вот пример парочки больших кластеров

In [6]:
print([c for c in all_clusters if len(c) > 4][:3])

[['Basf India Ltd.', 'Basf Japan Ltd. 6 10 1 Roppongi', 'Basf India', 'Basf Construction Chemicals Ua', 'Basf New Zealand Ltd.', 'Basf Sa', 'Basf Chile S.A.', 'Basf Construction Chemicals', 'Basf Corp.', 'Bdp International Basf Imp.', 'Basf Turk Kimya San Ve Tic.Tld.Sti', 'Basf Auxiliary Chemicals', 'Basf Turk Kimya San.Ve Tic Ltd.Sti', 'Basf Peruana S.A.', "Basf's Paper Chemicals (Huizhou) Co., Ltd.", 'Basf De Costa Rica Sociedad Anonima', 'Basf Japan Ltd.', 'Basf Co., Ltd. Yeosu', 'Basf Turk Kimya San. Ve Tic.Sti', 'Basf Quimica Colombiana S.A.', 'Basf Bangladesh Ltd.', 'Basf Pakistan Ltd.', 'Basf Co., Ltd.', 'Basf Colors & Effects Usa Llc', 'Basf Mexicana S.A. De C.V.', 'Basf Corporation', 'Basf Finlay Pvt., Ltd.', 'Basf (China) Co., Ltd. Shanghai', 'Basf Chile S A', 'Basf Pakistan (Private) Ltd.'], ['Pirelli Neumaticos S.A. De Cv', 'Pirelli Neumaticos Sa De Cv.', 'Pirelli De Venezuela C.A.', 'Pirelli Neumaticos Sa', 'Pirelli Neumaticos S.A.I.C.', 'Pirelli Neumaticos Argentina Sa', 

# 2. Датасеты

Все данные представлены в датафрейме `df`. Для теста возьмём 20%.

In [7]:
n_test_samples = round(df[df.is_duplicate == 1].shape[0] * 0.2)

n_balanced_samples = df[df.is_duplicate == 1].shape[0]

df_test = pd.concat(
    [
        df[df.is_duplicate == 1].iloc[:n_test_samples, :],
        df[df.is_duplicate == 0].iloc[:n_test_samples, :],
    ]
)

df_test.to_csv("test_balanced_dataset.csv", index=None)

df_train = pd.concat(
    [
        df[df.is_duplicate == 1].iloc[n_test_samples:, :],
        df[df.is_duplicate == 0].iloc[n_test_samples:, :],
    ]
)
df_train.to_csv("train_unbalanced_dataset.csv", index=None)

df_train_balanced = pd.concat(
    [
        df[df.is_duplicate == 1].iloc[n_test_samples:, :],
        df[df.is_duplicate == 0].iloc[n_test_samples:n_balanced_samples, :],
    ]
)
df_train_balanced.to_csv("train_balanced_dataset.csv", index=None)

По итогу получилось:
* Сбалансированный test датасет `test_balanced_dataset.csv`
* Несбалансированный train датасет, но зато полный `train_unbalanced_dataset.csv`
* Сбалансированный test датасет `train_balanced_dataset.csv`

# 3. Введение мертики map@n не представляется возможным и вот посему:

...из-за того, что задача под этой метрикой переходит из задачи классификации, в задачу ранжирования.
Для метрики, допустим, map@5 нужно иметь список из 5 наиболее релевантных компаний.
Поскольку во многих группах куда меньше экземпляров, я не понимаю как это можно реализовать.

map@n может применяться в поисковой выдаче, в рекомендательных системах, но в данном случае вопрос как это реализовать остаётся открытым

Предлагаю до тех пор использовать f1 score

# 4. Обучим классификатор

Для примера генерации фичей возьмём 2 имени из датасета

In [8]:
name_1, name_2 = df_train.iloc[0, :][:2]

In [9]:
name_1, name_2

('Bridgestone Hosepower Llc', 'Bridgestone(Shenyang) Tire Co., Ltd.')

1. Расстояние леввинштейна

In [10]:
#!pip install python-Levenshtein==0.12.0

In [11]:
import Levenshtein

In [12]:
df_train["levenshtein"] = [
    Levenshtein.distance(n1, n2)
    for n1, n2 in zip(df_train.name_1.values, df_train.name_2.values)
]

2. Расстояние из библиотеки fuzzywuzzy

In [13]:
#!pip install fuzzywuzzy

In [14]:
from fuzzywuzzy import fuzz

In [15]:
df_train["ratio"] = [
    fuzz.ratio(n1, n2) for n1, n2 in zip(df_train.name_1.values, df_train.name_2.values)
]
df_train["partial_ratio"] = [
    fuzz.partial_ratio(n1, n2)
    for n1, n2 in zip(df_train.name_1.values, df_train.name_2.values)
]
df_train["token_sort_ratio"] = [
    fuzz.token_sort_ratio(n1, n2)
    for n1, n2 in zip(df_train.name_1.values, df_train.name_2.values)
]
df_train["token_set_ratio"] = [
    fuzz.token_set_ratio(n1, n2)
    for n1, n2 in zip(df_train.name_1.values, df_train.name_2.values)
]
df_train["wratio"] = [
    fuzz.WRatio(n1, n2)
    for n1, n2 in zip(df_train.name_1.values, df_train.name_2.values)
]

In [16]:
X_train = df_train.iloc[:, 3:].values
y_train = df_train.is_duplicate.values

In [17]:
def create_features(df):
    df["levenshtein"] = [
        Levenshtein.distance(n1, n2)
        for n1, n2 in zip(df.name_1.values, df.name_2.values)
    ]
    df["ratio"] = [
        fuzz.ratio(n1, n2) for n1, n2 in zip(df.name_1.values, df.name_2.values)
    ]
    df["partial_ratio"] = [
        fuzz.partial_ratio(n1, n2) for n1, n2 in zip(df.name_1.values, df.name_2.values)
    ]
    df["token_sort_ratio"] = [
        fuzz.token_sort_ratio(n1, n2)
        for n1, n2 in zip(df.name_1.values, df.name_2.values)
    ]
    df["token_set_ratio"] = [
        fuzz.token_set_ratio(n1, n2)
        for n1, n2 in zip(df.name_1.values, df.name_2.values)
    ]
    df["wratio"] = [
        fuzz.WRatio(n1, n2) for n1, n2 in zip(df.name_1.values, df.name_2.values)
    ]

In [18]:
create_features(df_test)

In [19]:
X_test = df_test.iloc[:, 3:].values
y_test = df_test.is_duplicate.values

In [20]:
reg = xgb.XGBRegressor(n_estimators=1000)
_ = reg.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds=50,
    verbose=False,
)

/home/user/miniconda3/envs/matching/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [21]:
prediction = reg.predict(X_test)

In [22]:
best_tresh, best_f1 = 0, 0
for thresh in tqdm(range(0, 100)):
    thresh /= 100
    pred = [1 if x > thresh else 0 for x in prediction]
    f1 = f1_score(y_test, pred)
    if f1 > best_f1:
        best_tresh = thresh

100%|██████████| 100/100 [00:00<00:00, 507.08it/s]


In [23]:
print(best_tresh)

0.64


In [24]:
pred = [1 if x > best_tresh else 0 for x in prediction]

In [25]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.52      1.00      0.68       491
           1       1.00      0.06      0.11       491

    accuracy                           0.53       982
   macro avg       0.76      0.53      0.40       982
weighted avg       0.76      0.53      0.40       982



In [26]:
f1_score(y_test, pred)

0.11153846153846154

In [31]:
from sklearn.metrics import roc_auc_score

In [32]:
roc_auc_score(y_test, pred)

0.5295315682281059

### F1-score по этому способу: **0.11**
### ROC-AUC: **0.53**

# Сравнение со старым решением

In [33]:
import pickle

In [34]:
with open("char_mapper.pkl", "rb") as f:
    char_mapper = pickle.load(f)

In [35]:
df_test = pd.read_csv("test_balanced_dataset.csv")
df_train = pd.read_csv("train_unbalanced_dataset.csv")

In [36]:
def get_features_targets(df):
    lowercase = lambda string: string.lower()

    name_1 = [i.lower() for i in df.name_1.values]
    name_2 = [i.lower() for i in df.name_2.values]

    max_len = max(list(map(len, name_1 + name_2)))

    # let the blank be 255 chars len
    # 164 to map empty cells
    name_1_blank = [max(char_mapper.values()) + 1] * 255
    name_2_blank = [max(char_mapper.values()) + 1] * 255

    def name_to_embedding(name: str) -> list:
        embedding = [max(char_mapper.values()) + 1] * 255
        for i, char in enumerate(name):
            embedding[i] = char_mapper[char]
        return embedding

    embeddings_list_1 = [name_to_embedding(name) for name in name_1]
    embeddings_list_2 = [name_to_embedding(name) for name in name_2]

    features = [e1 + e2 for e1, e2 in zip(embeddings_list_1, embeddings_list_2)]
    features = np.array(features)

    targets = df.is_duplicate.values
    return features, targets

In [37]:
import numpy as np

In [38]:
X_train, y_train = get_features_targets(df_train)

In [39]:
X_test, y_test = get_features_targets(df_test)

In [40]:
reg = xgb.XGBRegressor(n_estimators=1000)

In [41]:
_ = reg.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds=50,
    verbose=False,
)

/home/user/miniconda3/envs/matching/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [42]:
prediction = reg.predict(X_test)

In [43]:
best_tresh, best_f1 = 0, 0
for thresh in tqdm(range(0, 100)):
    thresh /= 100
    pred = [1 if x > thresh else 0 for x in prediction]
    f1 = f1_score(y_test, pred)
    if f1 > best_f1:
        best_tresh = thresh

100%|██████████| 100/100 [00:00<00:00, 513.12it/s]


In [44]:
pred = [1 if x > best_tresh else 0 for x in prediction]

In [45]:
f1_score(y_test, pred)

0.4313099041533546

In [46]:
roc_auc_score(y_test, pred)

0.6374745417515275

### F1-score по этому способу: **0.43**
### ROC-AUC: **0.64**